In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore')

In [ ]:
os.chdir(r'/content/drive/Shareddrives/DATA 245 ML/Submission')

In [ ]:
import os
n_cpu = os.cpu_count()
print("Number of CPUs in the system:", n_cpu)

In [ ]:
# !nvidia-smi

In [ ]:
train= pd.read_csv('data/train.csv')
test= pd.read_csv('data/test.csv')
print(train.columns)

In [ ]:
train_data= train[['submitted_via_le', 'timely_response_le', 'year', 'month', 'day',
       'product_le', 'sub_product_le', 'issue_le', 'sub_issue_le',
       'company_ce', 'state_le', 'consumer_consent_provided_le']]

train_labels= train[['company_response_to_consumer_le']]

test_data= train[['submitted_via_le', 'timely_response_le', 'year', 'month', 'day',
       'product_le', 'sub_product_le', 'issue_le', 'sub_issue_le',
       'company_ce', 'state_le', 'consumer_consent_provided_le']]

test_labels= train[['company_response_to_consumer_le']]

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.metrics import accuracy_score, precision_score, f1_score, classification_report, make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Define StratifiedKFold with shuffle enabled
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
skf

In [ ]:
# Define scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision_weighted',
    'f1': 'f1_weighted'
}

In [ ]:
# Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()
dt_scores = cross_validate(dt_classifier, train_data, train_labels, cv=skf,
                           scoring=scoring, return_train_score=False)

print("Decision Tree Classifier 10-fold CV before tuning:")
print("Accuracy: ", np.round(np.mean(dt_scores['test_accuracy']), 5))
print("Precision: ", np.round(np.mean(dt_scores['test_precision']), 5))
print("F1 Score: ", np.round(np.mean(dt_scores['test_f1']), 5))
print()
pd.DataFrame(dt_scores)

In [ ]:
# WITH CPU the results are as in the table. With GPU there is slight changes to 2 digit accuracy. Run this code without GPU to see PPT reported results.
print("Decision Tree Classifier Accuracy before tuning: ", np.round(np.mean(dt_scores['test_accuracy']), 5))
print("Decision Tree Classifier Precision before tuning: ", np.round(np.mean(dt_scores['test_precision']),5))
print("Decision Tree Classifier F1 Score before tuning: ", np.round(np.mean(dt_scores['test_f1']),5))

In [ ]:
# LightGBM
lgbm_classifier = LGBMClassifier()
lgbm_scores = cross_validate(lgbm_classifier, train_data, train_labels, cv=skf,
                             scoring=scoring, return_train_score=False)

print("LightGBM 10-fold CV before tuning:")
print("Accuracy: ", np.round(np.mean(lgbm_scores['test_accuracy']), 5))
print("Precision: ", np.round(np.mean(lgbm_scores['test_precision']), 5))
print("F1 Score: ", np.round(np.mean(lgbm_scores['test_f1']), 5))
print()
pd.DataFrame(lgbm_scores)

In [ ]:
# XGBoost
xgb_classifier = XGBClassifier(n_jobs=-1)
xgb_scores = cross_validate(xgb_classifier, train_data, train_labels, cv=skf,
                            scoring=scoring, return_train_score=False)

print("XGBoost Classifier 10-fold CV before tuning:")
print("Accuracy: ", np.round(np.mean(xgb_scores['test_accuracy']), 5))
print("Precision: ", np.round(np.mean(xgb_scores['test_precision']), 5))
print("F1 Score: ", np.round(np.mean(xgb_scores['test_f1']), 5))
print()
pd.DataFrame(xgb_scores)

In [ ]:
# K-Nearest Neighbors (KNN)
knn_classifier = KNeighborsClassifier()
knn_scores = cross_validate(knn_classifier, train_data, train_labels, cv=skf,
                            scoring=scoring, return_train_score=False)

print("KNN Classifier 10-fold CV before tuning:")
print("Accuracy: ", np.round(np.mean(knn_scores['test_accuracy']), 5))
print("Precision: ", np.round(np.mean(knn_scores['test_precision']), 5))
print("F1 Score: ", np.round(np.mean(knn_scores['test_f1']), 5))
print()
pd.DataFrame(knn_scores)

In [ ]:
# Random Forest Classifier
rf_classifier = RandomForestClassifier(n_jobs=-1)
rf_scores = cross_validate(
    rf_classifier, train_data, train_labels, cv=skf,
    scoring=scoring, return_train_score=False)

print("Random Forest Classifier 10-fold CV before tuning:")
print("Accuracy: ", np.round(np.mean(rf_scores['test_accuracy']), 5))
print("Precision: ", np.round(np.mean(rf_scores['test_precision']), 5))
print("F1 Score: ", np.round(np.mean(rf_scores['test_f1']), 5))
print()
pd.DataFrame(rf_scores)

In [ ]:
# SVM with linear kernel
svm_classifier = SVC(kernel='linear')
svm_scores = cross_validate(svm_classifier, train_data, train_labels, cv=skf,
                            scoring=scoring, return_train_score=False)

In [ ]:
print("SVM (Linear Kernel) 10-fold CV before tuning:")
print("Accuracy: ", np.round(np.mean(svm_scores['test_accuracy']), 5))
print("Precision: ", np.round(np.mean(svm_scores['test_precision']), 5))
print("F1 Score: ", np.round(np.mean(svm_scores['test_f1']), 5))
print()
pd.DataFrame(svm_scores)

# hyper parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
dt_param_grid = {
    'max_depth': [3, 7, 10, 12],
    'min_samples_split': [2, 5, 10, 15]
}

rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 7, 10]
}

xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 7, 10],
    'learning_rate': [0.1, 0.01, 0.001]
}

svm_param_grid = {
    'C': [1, 10, 100],
}

lgbm_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.1, 0.01, 0.001]
}

knn_param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance']
}

In [ ]:
dt_grid_search = GridSearchCV(dt_classifier, dt_param_grid, cv=10)
dt_grid_search.fit(train_data, train_labels)

In [ ]:
dt_grid_search.best_params_

In [ ]:
dt_grid_search.best_score_

In [ ]:
rf_grid_search = GridSearchCV(rf_classifier, rf_param_grid, cv=5)
rf_grid_search.fit(train_data, train_labels)

In [ ]:
rf_grid_search.best_params_

In [ ]:
xgb_grid_search = GridSearchCV(xgb_classifier, xgb_param_grid, cv=5)
xgb_grid_search.fit(train_data, train_labels)

In [ ]:
xgb_grid_search.best_params_

In [ ]:
svm_grid_search = GridSearchCV(svm_classifier, svm_param_grid, cv=5)
svm_grid_search.fit(train_data, train_labels)

In [ ]:
svm_grid_search.best_params_

In [ ]:
lgbm_grid_search = GridSearchCV(lgbm_classifier, lgbm_param_grid, cv=5)
lgbm_grid_search.fit(train_data, train_labels)

In [ ]:
lgbm_grid_search.best_params_

In [ ]:
knn_grid_search = GridSearchCV(knn_classifier, knn_param_grid, cv=5)
knn_grid_search.fit(train_data, train_labels)

In [ ]:
knn_grid_search.best_params_

# Save the best performing tuned model for deploying

In [ ]:
from sklearn.externals import joblib

best_lgbm_classifier = lgbm_grid_search.best_estimator_

# Save the best performing model
joblib.dump(best_lgbm_classifier, 'LightGBM.pkl')